# Creating Knowledge graphs from Pdf files

In [11]:
from langchain.graphs import Neo4jGraph

url = ""
username =""
password = ""
graph = Neo4jGraph(
    url=url,
    username=username,
    password=password
)

In [12]:
import os
from dotenv import load_dotenv 

load_dotenv()

# Add OpenAI key
#os.environ['OPENAI_API_KEY']=''
api_key = os.environ['OPENAI_API_KEY']


In [13]:
from langchain_community.graphs.graph_document import (
    Node as BaseNode,
    Relationship as BaseRelationship,
    GraphDocument,
)
from langchain.schema import Document
from typing import List, Dict, Any, Optional
from langchain.pydantic_v1 import Field, BaseModel

class Property(BaseModel):
  """A single property consisting of key and value"""
  key: str = Field(..., description="key")
  value: str = Field(..., description="value")

class Node(BaseNode):
    properties: Optional[List[Property]] = Field(
        None, description="List of node properties")

class Relationship(BaseRelationship):
    properties: Optional[List[Property]] = Field(
        None, description="List of relationship properties"
    )

# class Source(BaseSource):
#     properties: Optional[List[Property]] = Field(
#         None, description="List of sources"
#     )        
        
class KnowledgeGraph(BaseModel):
    """Generate a knowledge graph with entities and relationships."""
    nodes: List[Node] = Field(
        ..., description="List of nodes in the knowledge graph")
    rels: List[Relationship] = Field(
        ..., description="List of relationships in the knowledge graph"
    )

In [14]:
def format_property_key(s: str) -> str:
    words = s.split()
    if not words:
        return s
    first_word = words[0].lower()
    capitalized_words = [word.capitalize() for word in words[1:]]
    return "".join([first_word] + capitalized_words)

def props_to_dict(props) -> dict:
    """Convert properties to a dictionary."""
    properties = {}
    if not props:
      return properties
    for p in props:
        properties[format_property_key(p.key)] = p.value
    return properties

def map_to_base_node(node: Node) -> BaseNode:
    """Map the KnowledgeGraph Node to the base Node."""
    properties = props_to_dict(node.properties) if node.properties else {}
    # Add name property for better Cypher statement generation
    properties["name"] = node.id.title()
    return BaseNode(
        id=node.id.title(), type=node.type.capitalize(), properties=properties
    )


def map_to_base_relationship(rel: Relationship) -> BaseRelationship:
    """Map the KnowledgeGraph Relationship to the base Relationship."""
    source = map_to_base_node(rel.source)
    target = map_to_base_node(rel.target)
    properties = props_to_dict(rel.properties) if rel.properties else {}
    return BaseRelationship(
        source=source, target=target, type=rel.type, properties=properties
    )

# Using GPT-3.5-turbo-16k model to create the knowledge graph

In [15]:
import os
from langchain.chains.openai_functions import (
    create_openai_fn_chain,
    create_structured_output_chain,
)
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

# os.environ["OPENAI_API_KEY"] = "sk-"
llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

def get_extraction_chain(
    allowed_nodes: Optional[List[str]] = None,
    allowed_rels: Optional[List[str]] = None
    ):
    prompt = ChatPromptTemplate.from_messages(
        [(
          "system",
          f"""# Knowledge Graph Instructions for GPT-4
## 1. Overview
You are a top-tier algorithm designed for extracting information in structured formats to build a knowledge graph.
- **Nodes** represent entities and concepts. They're akin to Wikipedia nodes.
- The aim is to achieve simplicity and clarity in the knowledge graph, making it accessible for a vast audience.
## 2. Labeling Nodes
- **Consistency**: Ensure you use basic or elementary types for node labels.
  - For example, when you identify an entity representing a person, always label it as **"person"**. Avoid using more specific terms like "mathematician" or "scientist".
- **Node IDs**: Never utilize integers as node IDs. Node IDs should be names or human-readable identifiers found in the text.
{'- **Allowed Node Labels:**' + ", ".join(allowed_nodes) if allowed_nodes else ""}
{'- **Allowed Relationship Types**:' + ", ".join(allowed_rels) if allowed_rels else ""}
## 3. Handling Numerical Data and Dates
- Numerical data, like age or other related information, should be incorporated as attributes or properties of the respective nodes.
- **No Separate Nodes for Dates/Numbers**: Do not create separate nodes for dates or numerical values. Always attach them as attributes or properties of nodes.
- **Property Format**: Properties must be in a key-value format.
- **Quotation Marks**: Never use escaped single or double quotes within property values.
- **Naming Convention**: Use camelCase for property keys, e.g., `birthDate`.
## 4. Coreference Resolution
- **Maintain Entity Consistency**: When extracting entities, it's vital to ensure consistency.
If an entity, such as "John Doe", is mentioned multiple times in the text but is referred to by different names or pronouns (e.g., "Joe", "he"),
always use the most complete identifier for that entity throughout the knowledge graph. In this example, use "John Doe" as the entity ID.
Remember, the knowledge graph should be coherent and easily understandable, so maintaining consistency in entity references is crucial.
## 5. Strict Compliance
Adhere to the rules strictly. Non-compliance will result in termination.
          """),
            ("human", "Use the given format to extract information from the following input: {input}"),
            ("human", "Tip: Make sure to answer in the correct format"),
        ])
    return create_structured_output_chain(KnowledgeGraph, llm, prompt, verbose=False)

In [16]:
def extract_and_store_graph(
    document: Document,
    nodes:Optional[List[str]] = None,
    rels:Optional[List[str]]=None) -> None:
    # Extract graph data using OpenAI functions
    extract_chain = get_extraction_chain(nodes, rels)
    data = extract_chain.invoke(document.page_content)['function']
    # Construct a graph document
    graph_document = GraphDocument(
      nodes = [map_to_base_node(node) for node in data.nodes],
      relationships = [map_to_base_relationship(rel) for rel in data.rels],
      source = document
    )
    # Store information into a graph
    print(graph_document)
    graph.add_graph_documents([graph_document])
    return graph_document

# With 200 chunk size

In [23]:

from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from datetime import datetime

loader = PyPDFLoader("../data/Apple stock during pandemic.pdf")
start_time = datetime.now()
pages = loader.load_and_split()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=200, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)

In [24]:
from tqdm import tqdm

distinct_nodes = set()
nodes = []
relations = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    graph_document=extract_and_store_graph(chunk)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        nodes.append(node.id)
        distinct_nodes.add(node.id)
        
    
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)
        
print("nodes list :", len(nodes))
print("Distinct nodes :",len(distinct_nodes))
print("Relationships :",len(relations))


  0%|          | 0/23 [00:00<?, ?it/s]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple'}), Node(id='Stockprice', type='Concept', properties={'name': 'Stockprice'}), Node(id='Epidemic', type='Event', properties={'name': 'Epidemic'})] relationships=[Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Stockprice', type='Concept', properties={'name': 'Stockprice'}), type='has'), Relationship(source=Node(id='Stockprice', type='Concept', properties={'name': 'Stockprice'}), target=Node(id='Epidemic', type='Event', properties={'name': 'Epidemic'}), type='during')] source=Document(page_content="The Changes of Apple’s Stock Price During the \nPandemic  \nWanting Sun1,* \n1WendySun, Kinglee High School, Zheng Zhou, He Nan, China, 450000  \n*Corresponding author. Email:  wendysun040718@163.com  \nABSTRACT  \nDuring the epidemic, the economies of various countries, including many large companies, were subject to very large \nfluctuations and impacts. Therefore, this arti

  4%|▍         | 1/23 [00:04<01:46,  4.85s/it]

nodes=[Node(id='Economy', type='Concept', properties={'name': 'Economy'}), Node(id='Apple', type='Company', properties={'name': 'Apple'}), Node(id='Stock Price', type='Concept', properties={'name': 'Stock Price'}), Node(id='Covid-19', type='Concept', properties={'name': 'Covid-19'})] relationships=[Relationship(source=Node(id='Economy', type='Concept', properties={'name': 'Economy'}), target=Node(id='Apple', type='Company', properties={'name': 'Apple'}), type='has been damaged'), Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Stock Price', type='Concept', properties={'name': 'Stock Price'}), type='fell sharply'), Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Covid-19', type='Concept', properties={'name': 'Covid-19'}), type='impacted'), Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Stock Price', type='Concept', properties={'name

  9%|▊         | 2/23 [00:13<02:34,  7.37s/it]

...

nodes=[Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Topic', properties={'name': 'Economic Activity On The Spread Of Covid-19 In The Philippines'}), Node(id='Insights From Apple And Google Mobility Indicators', type='Article', properties={'source': 'The Journal of Asian Finance, Economics and Business | Korea Science', 'link': 'https://www.koreascience.or.kr/article/JAKO202034651879125.page', 'name': 'Insights From Apple And Google Mobility Indicators'}), Node(id='Comparing Apple To Amazon: Just A Matter Of Words', type='Article', properties={'author': 'Shivaani, M. V.', 'source': 'Journal of Emerging Technologies in Accounting', 'link': 'https://meridian.allenpress.com/jeta/article-abstract/doi/10.2308/JETA-2020-045/464597/Comparing-Apple-to-Amazon-Just-a-Matter-of-Words', 'name': 'Comparing Apple To Amazon: Just A Matter Of Words'})] relationships=[Relationship(source=Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Top

 96%|█████████▌| 22/23 [03:57<00:09,  9.00s/it]

nodes=[Node(id='Apple', type='Company', properties={'ticker': 'AAPL', 'name': 'Apple'}), Node(id='Microsoft', type='Company', properties={'ticker': 'MSFT', 'name': 'Microsoft'})] relationships=[Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Microsoft', type='Company', properties={'name': 'Microsoft'}), type='competitor')] source=Document(page_content='aring -Apple -to-Amazon -Just-a-\nMatter -of-Words  \n[6] Apple (AAPL) Vs Microsoft (MSFT): Which Is A \nBetter Tech Stock To Buy Right Now?, 28 Jan. \n2021, www.nasdaq.com/articles/apple -aapl-vs-\nmicrosoft -msft%3A -which -is-a-better -tech-stock-\nto-buy-right -now-2021 -01-28. \n[7] Osipovich Alexander. “Apple’s market capitalisation \nhas fallen below trillion, and Microsoft is the only \nremaining U.S. company in the trillion -dollar \nclub. ” Wall Street News, 24 Mar. 2020, \ncn.wsj.com/articles /  \nAdvances in Economics, Business and Management Research, volume 203\n376', meta

100%|██████████| 23/23 [04:01<00:00, 10.52s/it]

nodes list : 111
Distinct nodes : 80
Relationships : 113


# With 500 chunk size

In [17]:
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from datetime import datetime
from tqdm import tqdm

loader = PyPDFLoader("../data/Apple stock during pandemic.pdf")
start_time = datetime.now()
pages = loader.load_and_split()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=500, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)

distinct_nodes = set()
nodes = []
relations = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    graph_document=extract_and_store_graph(chunk)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        nodes.append(node.id)
        distinct_nodes.add(node.id)
         
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)
        
print("nodes list :", len(nodes))
print("Distinct nodes :",len(distinct_nodes))
print("Relationships :",len(relations))        

  0%|          | 0/10 [00:00<?, ?it/s]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple'}), Node(id='Covid-19', type='Event', properties={'name': 'Covid-19'})] relationships=[Relationship(source=Node(id='Apple', type='Company', properties={'name': 'Apple'}), target=Node(id='Covid-19', type='Event', properties={'name': 'Covid-19'}), type='stock_price_changes', properties={'description': "During the pandemic, Apple's stock price experienced fluctuations."})] source=Document(page_content="The Changes of Apple’s Stock Price During the \nPandemic  \nWanting Sun1,* \n1WendySun, Kinglee High School, Zheng Zhou, He Nan, China, 450000  \n*Corresponding author. Email:  wendysun040718@163.com  \nABSTRACT  \nDuring the epidemic, the economies of various countries, including many large companies, were subject to very large \nfluctuations and impacts. Therefore, this article will explore the ups and downs of Apple's stock price duri ng the \nepidemic. At different points in time and severity, Apple's stock price has ris

 10%|█         | 1/10 [00:04<00:44,  4.93s/it]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple', 'specializesin': 'consumer electronics, computer software, online services', 'headquarters': 'Cupertino, California', 'founded': 'April 1, 1976', 'marketvalue': '$623.5 billion (2012)', 'largestcompanybymarketcap': 'Yes', 'mostvaluablebrand': 'Yes', 'marketvalue2020': '$2 trillion', 'fortuneglobal500rank': '6th (2021)', 'largesttechcompanybyrevenue': 'Yes', 'largestpcvendor': '4th (2021)', 'largestsmartphonemanufacturer': '4th (2021)'})] relationships=[] source=Document(page_content=' is the main source of power to \npromote world gr owth, once the Chinese economy is \naffected, the world economy will inevitably be affected. \nThus, in such an environment where the whole world is \naffected, to what extent have the industry giants that \ndominate the world been hit? Let\'s take the most \nfamilia r Apple company as an example, and analyze \nhow much impact this devastating epidemic has brought \nto Apple’s stocks in s

 20%|██        | 2/10 [00:13<00:54,  6.81s/it]

...

nodes=[Node(id='Yan', type='Person', properties={'name': 'Yan'}), Node(id='Covid-19', type='Virus', properties={'name': 'Covid-19'}), Node(id='Stock Market', type='Concept', properties={'name': 'Stock Market'}), Node(id='Investing Strategies', type='Concept', properties={'name': 'Investing Strategies'}), Node(id='Camba', type='Person', properties={'name': 'Camba'}), Node(id='Philippines', type='Country', properties={'name': 'Philippines'}), Node(id='Apple', type='Company', properties={'name': 'Apple'}), Node(id='Google', type='Company', properties={'name': 'Google'}), Node(id='Journal Of Asian Finance', type='Journal', properties={'name': 'Journal Of Asian Finance'})] relationships=[Relationship(source=Node(id='Yan', type='Person', properties={'name': 'Yan'}), target=Node(id='Covid-19', type='Virus', properties={'name': 'Covid-19'}), type='research', properties={'title': 'Analysis of the Effect of COVID-19 on the Stock Market and Investing Strategies', 'date': '2020-03-28', 'source': '

 90%|█████████ | 9/10 [01:52<00:12, 12.73s/it]

nodes=[Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Topic', properties={'name': 'Economic Activity On The Spread Of Covid-19 In The Philippines'}), Node(id='Insights From Apple And Google Mobility Indicators', type='Article', properties={'author': 'Shivaani, M. V.', 'publication': 'The Journal of Asian Finance, Economics and Business | Korea Science', 'url': 'https://www.koreascience.or.kr/article/JAKO202034651879125.page', 'name': 'Insights From Apple And Google Mobility Indicators'}), Node(id='Comparing Apple To Amazon: Just A Matter Of Words', type='Article', properties={'author': 'Shivaani, M. V.', 'publication': 'Journal of Emerging Technologies in Accounting', 'url': 'https://meridian.allenpress.com/jeta/article-abstract/doi/10.2308/JETA-2020-045/464597/Comparing-Apple-to-Amazon-Just-a-Matter-of-Words', 'name': 'Comparing Apple To Amazon: Just A Matter Of Words'}), Node(id='Apple (Aapl)', type='Company', properties={'name': 'Apple (Aapl)'}), Nod

100%|██████████| 10/10 [02:02<00:00, 12.21s/it]

nodes list : 52
Distinct nodes : 41
Relationships : 54


# With 1000 chunk size

In [18]:
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from datetime import datetime

loader = PyPDFLoader("../data/Apple stock during pandemic.pdf")
start_time = datetime.now()
pages = loader.load_and_split()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)

distinct_nodes = set()
nodes = []
relations = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    graph_document=extract_and_store_graph(chunk)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        nodes.append(node.id)
        distinct_nodes.add(node.id)
         
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)

print("nodes list :", len(nodes))
print("Distinct nodes :",len(distinct_nodes))
print("Relationships :",len(relations))        

  0%|          | 0/6 [00:00<?, ?it/s]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple', 'location': 'Cupertino, California', 'founded': 'April 1, 1976', 'industry': 'Technology', 'revenue': 'USD $274.52 billion (2020)', 'marketcap': 'USD $2.43 trillion (2021)'})] relationships=[] source=Document(page_content='The Changes of Apple’s Stock Price During the \nPandemic  \nWanting Sun1,* \n1WendySun, Kinglee High School, Zheng Zhou, He Nan, China, 450000  \n*Corresponding author. Email:  wendysun040718@163.com  \nABSTRACT  \nDuring the epidemic, the economies of various countries, including many large companies, were subject to very large \nfluctuations and impacts. Therefore, this article will explore the ups and downs of Apple\'s stock price duri ng the \nepidemic. At different points in time and severity, Apple\'s stock price has risen and fallen. By observing the data on \nYahoo, we can summarize Apple\'s stock price changes during this period. At the beginning of the epidemic, the global \neconomy has b 

 17%|█▋        | 1/6 [00:04<00:24,  4.93s/it]

nodes=[Node(id='Macro_Perspective', type='Perspective', properties={'description': 'From a macro perspective, the decline in population, the decrease in spending, and the increase in inventories have caused many industries to stagnate. This is bound to bring a huge blow on the global economy to a serious extent.', 'name': 'Macro_Perspective'}), Node(id='Global_Economy', type='Concept', properties={'description': 'The global economy is expected to shrink by 3% in 2020 due to the new crown epidemic, with a more severe impact than the 2008-2009 financial crisis.', 'name': 'Global_Economy'}), Node(id='Economic_Survey', type='Publication', properties={'title': 'April 2020 Global Economic Survey', 'name': 'Economic_Survey'}), Node(id='Epidemic_Impact', type='Concept', properties={'description': 'The impact of the epidemic crisis on economic contraction is predicted to be three times that of the financial crisis.', 'name': 'Epidemic_Impact'}), Node(id='Global_Recession', type='Event', propert

 33%|███▎      | 2/6 [00:27<01:01, 15.28s/it]

...

nodes=[Node(id='Covid-19', type='Concept', properties={'name': 'Covid-19'}), Node(id='Stock Market', type='Concept', properties={'name': 'Stock Market'}), Node(id='Investing Strategies', type='Concept', properties={'name': 'Investing Strategies'}), Node(id='Yan', type='Person', properties={'name': 'Yan'}), Node(id='Https://Papers.Ssrn.Com/Sol3/Papers.Cfm?Abstract_Id=3563380', type='Source', properties={'name': 'Https://Papers.Ssrn.Com/Sol3/Papers.Cfm?Abstract_Id=3563380'}), Node(id='Camba, J. A. C.', type='Person', properties={'name': 'Camba, J. A. C.'}), Node(id='The Journal Of Asian Finance', type='Journal', properties={'name': 'The Journal Of Asian Finance'})] relationships=[Relationship(source=Node(id='Covid-19', type='Concept', properties={'name': 'Covid-19'}), target=Node(id='Stock Market', type='Concept', properties={'name': 'Stock Market'}), type='effect'), Relationship(source=Node(id='Covid-19', type='Concept', properties={'name': 'Covid-19'}), target=Node(id='Investing Strate

 83%|████████▎ | 5/6 [01:20<00:17, 17.56s/it]

nodes=[Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Topic', properties={'name': 'Economic Activity On The Spread Of Covid-19 In The Philippines'}), Node(id='Insights From Apple And Google Mobility Indicators', type='Journal', properties={'source': 'The Journal of Asian Finance, Economics and Business | Korea Science', 'link': 'https://www.koreascience.or.kr/article/JAKO202034651879125.page', 'name': 'Insights From Apple And Google Mobility Indicators'}), Node(id='Comparing Apple To Amazon: Just A Matter Of Words', type='Article', properties={'author': 'Shivaani, M. V.', 'journal': 'Journal of Emerging Technologies in Accounting', 'source': 'Machine Learning World', 'link': 'https://meridian.allenpress.com/jeta/article-abstract/doi/10.2308/JETA-2020-045/464597/Comparing-Apple-to-Amazon-Just-a-Matter-of-Words', 'name': 'Comparing Apple To Amazon: Just A Matter Of Words'}), Node(id='Apple (Aapl)', type='Company', properties={'name': 'Apple (Aapl)'}), Nod

100%|██████████| 6/6 [01:40<00:00, 16.68s/it]

nodes list : 37
Distinct nodes : 33
Relationships : 37


# With 2000 chunk size

In [19]:
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from datetime import datetime

loader = PyPDFLoader("../data/Apple stock during pandemic.pdf")
start_time = datetime.now()
pages = loader.load_and_split()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=2000, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)

distinct_nodes = set()
nodes = []
relations = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    graph_document=extract_and_store_graph(chunk)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        nodes.append(node.id)
        distinct_nodes.add(node.id)
         
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)

print("nodes list :", len(nodes))
print("Distinct nodes :",len(distinct_nodes))
print("Relationships :",len(relations))        

  0%|          | 0/5 [00:00<?, ?it/s]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple', 'location': 'Cupertino, California', 'founded': 'April 1, 1976', 'industry': 'Technology', 'revenue': 'USD $274.52 billion (2020)', 'marketcapitalization': 'USD $2.43 trillion (2021)'})] relationships=[] source=Document(page_content='The Changes of Apple’s Stock Price During the \nPandemic  \nWanting Sun1,* \n1WendySun, Kinglee High School, Zheng Zhou, He Nan, China, 450000  \n*Corresponding author. Email:  wendysun040718@163.com  \nABSTRACT  \nDuring the epidemic, the economies of various countries, including many large companies, were subject to very large \nfluctuations and impacts. Therefore, this article will explore the ups and downs of Apple\'s stock price duri ng the \nepidemic. At different points in time and severity, Apple\'s stock price has risen and fallen. By observing the data on \nYahoo, we can summarize Apple\'s stock price changes during this period. At the beginning of the epidemic, the global \neco

 20%|██        | 1/5 [00:04<00:19,  4.97s/it]

nodes=[Node(id='Macroperspective', type='Perspective', properties={'description': 'From a macro perspective, the decline in population, the decrease in spending, and the increase in inventories have caused many industries to stagnate. This is bound to bring a huge blow on the global economy to a serious extent.', 'name': 'Macroperspective'}), Node(id='Globaleconomy', type='Concept', properties={'description': 'The global economy is expected to shrink by 3% in 2020 due to the new crown epidemic. The impact will be much more severe than the 2008-2009 financial crisis.', 'name': 'Globaleconomy'}), Node(id='Economicsurvey', type='Publication', properties={'title': 'April 2020 Global Economic Survey', 'name': 'Economicsurvey'}), Node(id='Experts', type='Group', properties={'description': 'Some experts have predicted that the impact of the epidemic crisis on economic contraction will be three times that of the financial crisis.', 'name': 'Experts'}), Node(id='Recession', type='Event', proper

 40%|████      | 2/5 [00:21<00:34, 11.62s/it]

...

nodes=[Node(id='Microsoft', type='Company', properties={'stockpriceplummeted': '14.74%', 'largestsingledaydrop': 'April 24, 2000', 'stockpricefallen': '27%', 'name': 'Microsoft'}), Node(id='Monday', type='Day', properties={'name': 'Monday'}), Node(id='April 24, 2000', type='Date', properties={'name': 'April 24, 2000'}), Node(id='February 10, 2020', type='Date', properties={'name': 'February 10, 2020'}), Node(id='March 23', type='Date', properties={'name': 'March 23'}), Node(id='March 16', type='Date', properties={'name': 'March 16'}), Node(id='140.0', type='Number', properties={'name': '140.0'}), Node(id='135.42', type='Number', properties={'name': '135.42'}), Node(id='Apple', type='Company', properties={'lowestpricedate': 'March 23', 'openingprice': '140.0', 'closingprice': '135.42', 'marketvalue': 'higher than Microsoft', 'name': 'Apple'}), Node(id='August', type='Month', properties={'name': 'August'}), Node(id='November', type='Month', properties={'name': 'November'}), Node(id='Earl

 80%|████████  | 4/5 [00:46<00:13, 13.45s/it]

nodes=[Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Topic', properties={'name': 'Economic Activity On The Spread Of Covid-19 In The Philippines'}), Node(id='Insights From Apple And Google Mobility Indicators', type='Article', properties={'source': 'The Journal of Asian Finance, Economics and Business | Korea Science', 'link': 'https://www.koreascience.or.kr/article/JAKO202034651879125.page', 'name': 'Insights From Apple And Google Mobility Indicators'}), Node(id='Comparing Apple To Amazon: Just A Matter Of Words', type='Article', properties={'author': 'Shivaani, M. V.', 'source': 'Journal of Emerging Technologies in Accounting', 'link': 'https://meridian.allenpress.com/jeta/article-abstract/doi/10.2308/JETA-2020-045/464597/Comparing-Apple-to-Amazon-Just-a-Matter-of-Words', 'name': 'Comparing Apple To Amazon: Just A Matter Of Words'}), Node(id='Apple (Aapl) Vs Microsoft (Msft): Which Is A Better Tech Stock To Buy Right Now?', type='Article', properties

100%|██████████| 5/5 [00:58<00:00, 11.64s/it]

nodes list : 30
Distinct nodes : 26
Relationships : 17


# With 5000 chunk size

In [20]:
from langchain.text_splitter import TokenTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from datetime import datetime

loader = PyPDFLoader("../data/Apple stock during pandemic.pdf")
start_time = datetime.now()
pages = loader.load_and_split()
# Define chunking strategy
text_splitter = TokenTextSplitter(chunk_size=5000, chunk_overlap=20)
chunks = text_splitter.split_documents(pages)

distinct_nodes = set()
nodes = []
relations = []

for i, chunk in tqdm(enumerate(chunks), total=len(chunks)):
    graph_document=extract_and_store_graph(chunk)
    
    #Get distinct nodes
    for node in graph_document.nodes :
        nodes.append(node.id)
        distinct_nodes.add(node.id)
         
    #Get all relations   
    for relation in graph_document.relationships :
        relations.append(relation.type)

  0%|          | 0/5 [00:00<?, ?it/s]

nodes=[Node(id='Apple', type='Company', properties={'name': 'Apple', 'location': 'Cupertino, California', 'founded': 'April 1, 1976', 'industry': 'Technology', 'revenue': 'USD $274.52 billion (2020)', 'marketcapitalization': 'USD $2.43 trillion (2021)'})] relationships=[] source=Document(page_content='The Changes of Apple’s Stock Price During the \nPandemic  \nWanting Sun1,* \n1WendySun, Kinglee High School, Zheng Zhou, He Nan, China, 450000  \n*Corresponding author. Email:  wendysun040718@163.com  \nABSTRACT  \nDuring the epidemic, the economies of various countries, including many large companies, were subject to very large \nfluctuations and impacts. Therefore, this article will explore the ups and downs of Apple\'s stock price duri ng the \nepidemic. At different points in time and severity, Apple\'s stock price has risen and fallen. By observing the data on \nYahoo, we can summarize Apple\'s stock price changes during this period. At the beginning of the epidemic, the global \neco

 20%|██        | 1/5 [00:05<00:21,  5.38s/it]

nodes=[Node(id='Macroperspective', type='Perspective', properties={'description': 'From a macro perspective, the decline in population, the decrease in spending, and the increase in inventories have caused many industries to stagnate. This is bound to bring a huge blow on the global economy to a serious extent.', 'name': 'Macroperspective'}), Node(id='Globaleconomy', type='Economy', properties={'impact': "The International Monetary Fund stated in the 'April 2020 Global Economic Survey' that due to the new crown epidemic, the global economy is expected to shrink by 3% in 2020, and the impact will be much more severe than the 2008-2009 financial crisis.", 'name': 'Globaleconomy'}), Node(id='Epidemicimpact', type='Impact', properties={'comparisontofinancialcrisis': 'Some experts have predicted that the impact of the epidemic crisis on economic contraction will be three times that of the financial crisis.', 'name': 'Epidemicimpact'}), Node(id='Globalrecession', type='Recession', properties

 40%|████      | 2/5 [00:20<00:32, 10.84s/it]

...

 60%|██████    | 3/5 [00:24<00:15,  7.98s/it]

nodes=[Node(id='Microsoft', type='Company', properties={'stockpriceplummeted': '14.74%', 'largestsingledaydrop': 'April 24, 2000', 'stockpricefallen': '27%', 'name': 'Microsoft'}), Node(id='Monday', type='Day', properties={'name': 'Monday'}), Node(id='April 24, 2000', type='Date', properties={'name': 'April 24, 2000'}), Node(id='February 10, 2020', type='Date', properties={'name': 'February 10, 2020'}), Node(id='March 23', type='Date', properties={'name': 'March 23'}), Node(id='March 16', type='Date', properties={'name': 'March 16'}), Node(id='140.0', type='Price', properties={'name': '140.0'}), Node(id='135.42', type='Price', properties={'name': '135.42'}), Node(id='Apple', type='Company', properties={'lowestpricedate': 'March 23', 'openingprice': '140.0', 'closingprice': '135.42', 'marketvalue': 'higher than Microsoft', 'name': 'Apple'}), Node(id='August', type='Month', properties={'name': 'August'}), Node(id='November', type='Month', properties={'name': 'November'}), Node(id='Early 

 80%|████████  | 4/5 [00:44<00:12, 12.67s/it]

nodes=[Node(id='Economic Activity On The Spread Of Covid-19 In The Philippines', type='Topic', properties={'name': 'Economic Activity On The Spread Of Covid-19 In The Philippines'}), Node(id='Insights From Apple And Google Mobility Indicators', type='Article', properties={'source': 'The Journal of Asian Finance, Economics and Business | Korea Science', 'link': 'https://www.koreascience.or.kr/article/JAKO202034651879125.page', 'name': 'Insights From Apple And Google Mobility Indicators'}), Node(id='Comparing Apple To Amazon: Just A Matter Of Words', type='Article', properties={'author': 'Shivaani, M. V.', 'source': 'Journal of Emerging Technologies in Accounting', 'link': 'https://meridian.allenpress.com/jeta/article-abstract/doi/10.2308/JETA-2020-045/464597/Comparing-Apple-to-Amazon-Just-a-Matter-of-Words', 'name': 'Comparing Apple To Amazon: Just A Matter Of Words'}), Node(id='Apple (Aapl) Vs Microsoft (Msft): Which Is A Better Tech Stock To Buy Right Now?', type='Article', properties

100%|██████████| 5/5 [00:53<00:00, 10.74s/it]


In [21]:
print("nodes list :", len(nodes))
print("Distinct nodes :",len(distinct_nodes))
print("Relationships :",len(relations))

nodes list : 28
Distinct nodes : 24
Relationships : 16
